# MC Dropout

In [1]:
%load_ext autoreload

In [2]:
import os, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import  Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, Input
from tensorflow.keras.datasets import mnist
import tensorflow.keras.backend as K

In [3]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")

sys.path.append(MODULES_PATH)
from models import setup_growth

In [4]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [5]:
batch_size = 128
num_classes = 10
epochs = 12

# Input image dimensions
img_rows, img_cols = 28, 28

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [8]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

print("x_train shape: ", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
def get_dropout(input_tensor, p=0.5, mc=False):
    if mc:
        return Dropout(p)(input_tensor, training=True)
    else:
        return Dropout(p)(input_tensor)


def get_model(mc=False, act="relu"):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation=act)(inp)
    x = Conv2D(64, kernel_size=(3, 3), activation=act)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = get_dropout(x, p=0.25, mc=mc)
    x = Flatten()(x)
    x = Dense(128, activation=act)(x)
    x = get_dropout(x, p=0.5, mc=mc)
    out = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model


In [11]:
model = get_model()
mc_model = get_model(mc=True)

In [12]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 12s 26ms/step - loss: 2.2937 - accuracy: 0.1238 - val_loss: 2.2751 - val_accuracy: 0.2104
Epoch 2/10
469/469 [==============================] - 4s 7ms/step - loss: 2.2630 - accuracy: 0.2062 - val_loss: 2.2393 - val_accuracy: 0.3608
Epoch 3/10
469/469 [==============================] - 3s 7ms/step - loss: 2.2258 - accuracy: 0.2959 - val_loss: 2.1922 - val_accuracy: 0.4370
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 2.1756 - accuracy: 0.3720 - val_loss: 2.1265 - val_accuracy: 0.5449
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 2.1020 - accuracy: 0.4453 - val_loss: 2.0333 - val_accuracy: 0.6373
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 2.0016 - accuracy: 0.5001 - val_loss: 1.9031 - val_accuracy: 0.6913
Epoch 7/10
469/469 [==============================] - 3s 7ms/step - loss: 1.8669 - accuracy: 0.5441 - val_loss: 1.7283 - val_accuracy: 0.7198
Epoc

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)

print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Test loss:  1.1152267456054688
Test accuracy:  0.7894999980926514


In [14]:
h_mc = mc_model.fit(x_train, y_train, batch_size=batch_size, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 3s 7ms/step - loss: 2.2833 - accuracy: 0.1287 - val_loss: 2.2573 - val_accuracy: 0.1672
Epoch 2/10
469/469 [==============================] - 3s 7ms/step - loss: 2.2346 - accuracy: 0.2166 - val_loss: 2.2032 - val_accuracy: 0.2743
Epoch 3/10
469/469 [==============================] - 3s 7ms/step - loss: 2.1726 - accuracy: 0.3151 - val_loss: 2.1327 - val_accuracy: 0.3538
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 2.0896 - accuracy: 0.3971 - val_loss: 2.0351 - val_accuracy: 0.4411
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 1.9841 - accuracy: 0.4629 - val_loss: 1.9122 - val_accuracy: 0.4972
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 1.8459 - accuracy: 0.5239 - val_loss: 1.7589 - val_accuracy: 0.5492
Epoch 7/10
469/469 [==============================] - 3s 7ms/step - loss: 1.6831 - accuracy: 0.5715 - val_loss: 1.5887 - val_accuracy: 0.5989
Epoch 

# Debugg Mc Dropout

In [28]:
%autoreload 2
from active_learning import TrainConfig
from bayesian import McDropout

In [223]:
mc_model.fit(x_train, y_train, epochs=100, batch_size=60)

Epoch 1/100
1000/1000 [==============================] - 4s 4ms/step - loss: 1.0690 - accuracy: 0.6945
Epoch 2/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.9515 - accuracy: 0.7186
Epoch 3/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.8713 - accuracy: 0.7372
Epoch 4/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.8135 - accuracy: 0.7556
Epoch 5/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.7631 - accuracy: 0.7689
Epoch 6/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.7294 - accuracy: 0.7781
Epoch 7/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6928 - accuracy: 0.7891
Epoch 8/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6630 - accuracy: 0.7998
Epoch 9/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6432 - accuracy: 0.8040
Epoch 10/100
1000/1000 [==============================] - 4s 4ms/step - l

1000/1000 [==============================] - 4s 4ms/step - loss: 0.3085 - accuracy: 0.9086
Epoch 80/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3084 - accuracy: 0.9080
Epoch 81/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3089 - accuracy: 0.9089
Epoch 82/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3074 - accuracy: 0.9086
Epoch 83/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3057 - accuracy: 0.9096
Epoch 84/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3043 - accuracy: 0.9101
Epoch 85/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3048 - accuracy: 0.9117
Epoch 86/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.2980 - accuracy: 0.9105
Epoch 87/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.2963 - accuracy: 0.9120
Epoch 88/100
1000/1000 [==============================] - 4s 4ms/step - loss:

In [229]:
mc_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2722 - accuracy: 0.9203


[0.27217549085617065, 0.9203000068664551]